In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb

In [2]:
df= pd.read_excel("../data/raw/entrenamiento.xlsx")
df= df.rename(columns={"C7.1": "C8", "des": "QUALITY"})


In [4]:
high_values_train = df["C8"] > 900
df.loc[high_values_train, "C8"] = df.loc[high_values_train, "C8"] / 1000

In [5]:
def impute_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = (data[column] < lower_bound) | (data[column] > upper_bound)

    # Impute outliers with the median of the column
    data.loc[outliers, column] = data[column].median()
    return data


# Impute outliers in the dataset using IQR method
df_imputed_outliers = df.copy()
for column in df.columns:
    if df[column].dtype != 'object':  # Apply only for numerical columns
        df_imputed_outliers = impute_outliers_iqr(df_imputed_outliers, column)

In [6]:
# Impute missing values using median for predictor columns (excluding 'des' or 'QUALITY')
for column in df_imputed_outliers.columns:
    if column != "des" and df_imputed_outliers[
        column].dtype != 'object':  # Exclude target column and non-numerical columns
        median_value = df_imputed_outliers[column].median()
        df_imputed_outliers[column].fillna(median_value, inplace=True)

# Check if there are any missing values left
remaining_missing_values = df_imputed_outliers.isnull().sum()
remaining_missing_values[remaining_missing_values > 0]

Series([], dtype: int64)

In [8]:
df_completed = df_imputed_outliers.copy()
# Separar las características y la variable objetivo
X = df_completed.drop("QUALITY", axis=1)
y = df_completed["QUALITY"]

# Estandarizar las características
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
## Splitting the DataFrame
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_std, X_test_std, y_train, y_test = train_test_split(X_standardized, y, test_size=0.3, random_state=42)

In [9]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Paso 1: Definir el modelo
gb = GradientBoostingRegressor(random_state=42)

# Paso 2: Establecer un conjunto de hiperparámetros para buscar
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Paso 3: Usar GridSearchCV
grid_search = GridSearchCV(gb, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search.fit(X_train_std, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

# Paso 4: Entrenar el modelo
best_gb = grid_search.best_estimator_

# Paso 5: Evaluar el rendimiento del modelo
train_predictions = best_gb.predict(X_train_std)
mae = mean_absolute_error(y_train, train_predictions)
print(f"Mean Absolute Error (MAE) con hiperparámetros optimizados: {mae:.4f}")


Mejores hiperparámetros: {'learning_rate': 0.1, 'max_depth': 5, 'max_features': None, 'n_estimators': 200, 'subsample': 0.9}
Mean Absolute Error (MAE) con hiperparámetros optimizados: 0.2508


In [10]:
from sklearn.model_selection import cross_val_score

# Usando el modelo con los mejores hiperparámetros
scores = cross_val_score(best_gb, X_train_std, y_train, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

# Convertir a MAE positivo
mae_scores = -scores

# Mostrar los resultados de la validación cruzada
print(f"MAE scores from cross-validation: {mae_scores}")
print(f"Mean MAE score: {mae_scores.mean()}")
print(f"Standard Deviation of MAE scores: {mae_scores.std()}")


MAE scores from cross-validation: [0.496141   0.50923275 0.48737803 0.47747602 0.49874127]
Mean MAE score: 0.49379381392790067
Standard Deviation of MAE scores: 0.010731077455014906
